# Propuesta "*Alta Resolución Generative Adversarial Network*"
## Alumno: Patrick Xavier Marquez Choque
## Curso: Proyecto Final de Carrera II
## Periodo: 2023-I

In [ ]:
import tensorflow as tf
from tensorflow.keras.layers import Dense, Conv2D, Conv2DTranspose, LeakyReLU, BatchNormalization, Reshape, Flatten
from tensorflow.keras.models import Sequential

# Definir el generador
def build_generator():
    generator = Sequential([
        Dense(4 * 4 * 1024, input_shape=(100,)),
        Reshape((4, 4, 1024)),
        Conv2DTranspose(512, (5, 5), strides=(2, 2), padding='same', activation='relu'),
        BatchNormalization(),
        Conv2DTranspose(256, (5, 5), strides=(2, 2), padding='same', activation='relu'),
        BatchNormalization(),
        Conv2DTranspose(128, (5, 5), strides=(2, 2), padding='same', activation='relu'),
        BatchNormalization(),
        Conv2DTranspose(3, (5, 5), strides=(2, 2), padding='same', activation='tanh')
    ])
    return generator

# Definir el discriminador
def build_discriminator():
    discriminator = Sequential([
        Conv2D(64, (5, 5), strides=(2, 2), padding='same', input_shape=(512, 512, 3)),
        LeakyReLU(0.2),
        Conv2D(128, (5, 5), strides=(2, 2), padding='same'),
        BatchNormalization(),
        LeakyReLU(0.2),
        Conv2D(256, (5, 5), strides=(2, 2), padding='same'),
        BatchNormalization(),
        LeakyReLU(0.2),
        Conv2D(512, (5, 5), strides=(2, 2), padding='same'),
        BatchNormalization(),
        LeakyReLU(0.2),
        Flatten(),
        Dense(1, activation='sigmoid')
    ])
    return discriminator

# Construir el generador y el discriminador
generator = build_generator()
discriminator = build_discriminator()

# Definir los hiperparámetros
batch_size = 64
latent_dim = 100
epochs = 100

# Definir la función de pérdida y los optimizadores
loss_fn = tf.keras.losses.BinaryCrossentropy()
generator_optimizer = tf.keras.optimizers.Adam(learning_rate=0.0002, beta_1=0.5)
discriminator_optimizer = tf.keras.optimizers.Adam(learning_rate=0.0002, beta_1=0.5)

# Definir la función de entrenamiento
@tf.function
def train_step(real_images):
    # Generar ruido aleatorio
    noise = tf.random.normal((batch_size, latent_dim))

    with tf.GradientTape() as gen_tape, tf.GradientTape() as disc_tape:
        # Generar imágenes falsas con el generador
        fake_images = generator(noise, training=True)

        # Calcular la pérdida del discriminador en imágenes reales y falsas
        real_output = discriminator(real_images, training=True)
        fake_output = discriminator(fake_images, training=True)
        gen_loss = loss_fn(tf.ones_like(fake_output), fake_output)
        disc_loss = loss_fn(tf.ones_like(real_output), real_output) + loss_fn(tf.zeros_like(fake_output), fake_output)

    # Calcular los gradientes y aplicar las actualizaciones
    gradients_gen = gen_tape.gradient(gen_loss, generator.trainable_variables)
    gradients_disc = disc_tape.gradient(disc_loss, discriminator.trainable_variables)
    generator_optimizer.apply_gradients(zip(gradients_gen, generator.trainable_variables))
    discriminator_optimizer.apply_gradients(zip(gradients_disc, discriminator.trainable_variables))

# Cargar el dataset CelebA (Asegúrate de tener los archivos de imágenes en la carpeta "./celeba/img_align_celeba")
dataset = tf.keras.preprocessing.image_dataset_from_directory(
    "./celeba",
    image_size=(512, 512),
    batch_size=batch_size,
    validation_split=0.2,
    subset="training",
    seed=123
)

# Preprocesar el dataset y definir el número de pasos por época
dataset = dataset.map(lambda x: x / 255.0)
steps_per_epoch = len(dataset) // batch_size

# Entrenar el modelo
for epoch in range(epochs):
    print(f"Epoch {epoch+1}/{epochs}")
    for step, real_images in enumerate(dataset):
        train_step(real_images)

    # Guardar imágenes generadas
    random_noise = tf.random.normal((16, latent_dim))
    generated_images = generator(random_noise, training=False)
    generated_images = (generated_images + 1) * 0.5  # Escalar de -1 a 1 a 0 a 1
    for i in range(generated_images.shape[0]):
        image = tf.cast(generated_images[i] * 255, tf.uint8)
        tf.keras.preprocessing.image.save_img(f"generated_images/image_{epoch}_{i}.png", image)


In [ ]:
def calculate_psnr(image1, image2):
    mse = np.mean((image1 - image2) ** 2)
    max_value = np.max(image1)
    psnr = 20 * np.log10(max_value / np.sqrt(mse))
    return psnr


In [ ]:
psnr_values = []
for i in range(num_images):
    psnr = calculate_psnr(original_images[i], generated_images[i])
    psnr_values.append(psnr)
    print(f"PSNR for image {i+1}: {psnr} dB")

average_psnr = np.mean(psnr_values)
print(f"Average PSNR: {average_psnr} dB")


In [ ]:
import numpy as np
from skimage.metrics import peak_signal_noise_ratio

def calculate_psnr(original_images, generated_images):
    psnr_values = []
    for i in range(len(original_images)):
        psnr = peak_signal_noise_ratio(original_images[i], generated_images[i], data_range=1.0)
        psnr_values.append(psnr)
        print(f"PSNR for image {i+1}: {psnr} dB")

    average_psnr = np.mean(psnr_values)
    print(f"Average PSNR: {average_psnr} dB")

# Supongamos que tienes las imágenes originales cargadas en la variable 'original_images'
# y las imágenes generadas en la variable 'generated_images'

# Carga las imágenes originales de CIFAR-10
# (reemplaza 'load_cifar10' con el código necesario para cargar el conjunto de datos)
original_images = load_cifar10()

# Calcula el PSNR
calculate_psnr(original_images, generated_images)